In [ ]:
#@title Примонтирование Google Drive

FOLDER = "2022-IntroPytorch" 
PATH = "drive/MyDrive/" + FOLDER  + "/"
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append(PATH)

Mounted at /content/drive


## Подготавливаем среду

In [ ]:
# Установка нужных пакетов
!apt-get install -y xvfb python-opengl ffmpeg x11-utils > /dev/null 2>&1

In [ ]:
!pip install pettingzoo
!pip install multi-agent-ale-py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 816 kB 4.9 MB/s 
     |████████████████████████████████| 836 kB 40.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 778 kB 5.4 MB/s 


In [ ]:
!pip install autorom[accept-rom-license]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.4.2-py3-none-any.whl size=441028 sha256=5db838c04f3605ddbaf3cf0a1ea4093df504006a80cb4750ce7f016f5dc403cb
  Stored in directory: /root/.cache/pip/wheels/87/67/2e/6147e7912fe37f5408b80d07527dab807c1d25f5c403a9538a
Successfully built AutoROM.accept-rom-license


In [ ]:
!AutoROM

AutoROM will download the Atari 2600 ROMs.
They will be installed to:
	/usr/local/lib/python3.7/dist-packages/AutoROM/roms
	/usr/local/lib/python3.7/dist-packages/multi_agent_ale_py/roms

Existing ROMs will be overwritten.

I own a license to these Atari 2600 ROMs.
I agree to not distribute these ROMs and wish to proceed: [Y/n]: Y
Installed /usr/local/lib/python3.7/dist-packages/AutoROM/roms/adventure.bin
Installed /usr/local/lib/python3.7/dist-packages/multi_agent_ale_py/roms/adventure.bin
Installed /usr/local/lib/python3.7/dist-packages/AutoROM/roms/air_raid.bin
Installed /usr/local/lib/python3.7/dist-packages/multi_agent_ale_py/roms/air_raid.bin
Installed /usr/local/lib/python3.7/dist-packages/AutoROM/roms/alien.bin
Installed /usr/local/lib/python3.7/dist-packages/multi_agent_ale_py/roms/alien.bin
Installed /usr/local/lib/python3.7/dist-packages/AutoROM/roms/amidar.bin
Installed /usr/local/lib/python3.7/dist-packages/multi_agent_ale_py/roms/amidar.bin
Installed /usr/local/lib/python

In [ ]:
from IPython.display import clear_output, HTML
from IPython import display as ipythondisplay
import base64
import os
import io

def show_video(image_path):

  if not os.path.exists(f'{image_path}.mp4'):
    os.system(f"ffmpeg -framerate 60 -pattern_type glob -i '{image_path}/*.png' -c:v libx264 -pix_fmt yuv420p {image_path}.mp4")

  video = io.open(f'{image_path}.mp4', 'r+b').read()
  encoded = base64.b64encode(video)
  ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
          controls style="height: 400px;">
          <source src="data:video/mp4;base64,{0}" type="video/mp4" />
        </video>'''.format(encoded.decode('ascii'))))


# Работа со средой

In [ ]:
from pettingzoo.atari import boxing_v2

In [ ]:
env = boxing_v2.parallel_env()

In [ ]:
# Наблюдение - это 1 картинка
env.observation_spaces

/usr/local/lib/python3.7/dist-packages/pettingzoo/utils/conversions.py:92: UserWarning: The `observation_spaces` dictionary is deprecated. Use the `observation_space` function instead.
  "The `observation_spaces` dictionary is deprecated. Use the `observation_space` function instead."


{'first_0': Box(0, 255, (210, 160, 3), uint8),
 'second_0': Box(0, 255, (210, 160, 3), uint8)}

In [ ]:
# 18 действий
env.action_spaces

/usr/local/lib/python3.7/dist-packages/pettingzoo/utils/conversions.py:106: UserWarning: The `action_spaces` dictionary is deprecated. Use the `action_space` function instead.
  "The `action_spaces` dictionary is deprecated. Use the `action_space` function instead."


{'first_0': Discrete(18), 'second_0': Discrete(18)}

In [ ]:
def get_boxer_coordinate(image, player = "you"):
  if player == "you":
    mean_pos = (image[25:-25,25:-25,:] > 210).nonzero()
    return mean_pos[0].float().mean() + 25, mean_pos[1].float().mean() + 25
  else:
    mean_pos = (image[25:-25,25:-25,:] < 10).nonzero()
    return mean_pos[0].float().mean() + 25, mean_pos[1].float().mean() + 25


In [ ]:
import numpy as np

def invert_players(image):
  img = image[25:-25,25:-25,:]
  flags = img.sum(axis=2)
  img[flags == 0] = np.array([214]*3)
  img[flags == 214*3] = np.array([0,0,0])
  image[25:-25,25:-25,:] = img
  return image

def correct_observations(observations):
  observations["second_0"] = invert_players(observations["second_0"].copy())
  return observations


In [ ]:
import torch
from torch.distributions import Categorical
from torchvision.utils import save_image
import pandas as pd
import os

def play_with_agents(env, agents, max_steps = 200, device = None, render = True):
  if device:
    for name, agent in agents.items():
      agent.to(device)
    
  if render:
    os.system("mkdir -p images")
    os.system("rm -r images/*")
    
  frame = 0

  total_reward = {}
  render_result = {}
  trajectories = {agent_name:{'observations':[], 'actions':[], 'rewards':[]} for agent_name in agents.keys()}
 
  old_observations = correct_observations(env.reset())
  observations = correct_observations(env.reset())
 
  while env.agents:
    if render:
      # Сохраняем картинки для всех агентов
      for a in env.agents:
        agent_path = f'images/{a}/'
        if a not in render_result:
          render_result[a] = agent_path
          os.system(f"mkdir -p images/{a}")

        img = torch.tensor(observations[a],dtype=torch.float64)
        img /= 255.0
        img = img.permute(2,0,1)
        save_image(img, f'{agent_path}/frame{frame:05d}.png')

    frame += 1

    actions = {}
    for agent_name in env.agents:
      probs = agents[agent_name](
          torch.FloatTensor(observations[agent_name]).to(device))
      
      dist = Categorical(probs)
      action = dist.sample().item()

      actions[agent_name] = action

      trajectories[agent_name]['observations'].append(observations[agent_name])
      trajectories[agent_name]['actions'].append(action)
      picture = observations[agent_name]

    observations, rewards, terminations, truncations, infos = env.step(actions)

    observations = correct_observations(observations)

    '''
    Сравниваем расстояние между агентами на предыдущем кадре и на текущем
    Вознаграждаем, если оно увеличилось
    '''
    for agent_name in agents:
      new_me = get_boxer_coordinate(torch.from_numpy(observations[agent_name]), player = "you")
      new_them = get_boxer_coordinate(torch.from_numpy(observations[agent_name]), player = "them")
      old_me = get_boxer_coordinate(torch.from_numpy(old_observations[agent_name]), player = "you")
      old_them = get_boxer_coordinate(torch.from_numpy(old_observations[agent_name]), player = "them")
      if ((new_me[0] - new_them[0]) ** 2 + (new_me[1] - new_them[1]) ** 2) ** 0.5 < 10:
          total_reward[agent_name] = total_reward.get(agent_name, 0) + rewards[agent_name]
      elif ((new_me[0] - new_them[0]) ** 2 + (new_me[1] - new_them[1]) ** 2) ** 0.5 < ((old_me[0] - old_them[0]) ** 2 + (old_me[1] - old_them[1]) ** 2) ** 0.5:
        total_reward[agent_name] = total_reward.get(agent_name, 0) + rewards[agent_name] + 0.05
      else:
        total_reward[agent_name] = total_reward.get(agent_name, 0) + rewards[agent_name] - 0.01
      trajectories[agent_name]['rewards'].append(rewards[agent_name])


    if all([f for f in terminations.values()]):
      break
    if frame > max_steps:
      break

    old_observations = observations

  if render:
    #create_video("images/")
    os.system("ffmpeg -framerate 60 -pattern_type glob -i './images/*.png' -c:v libx264 -pix_fmt yuv420p out.mp4")

  return trajectories, total_reward, render_result


In [ ]:
import torch

class StunnedAgent:
  '''
  Агент просто стоит на месте
  '''
  def __init__(self):
    pass

  def __call__(self, state):

    # Агент должен вернуть вероятности каждого действия, а у нас их 18.
    # Бездействие -- действие номер ноль, туда помещаем 100%
    return torch.tensor([1.] + [0.]*17)

In [ ]:
import torch

class UnpredictableAgent:
  '''
  Агент выполняет случайное действие с равной вероятностью для каждого
  '''
  def __init__(self):
    pass

  def __call__(self, state):
    return torch.tensor([1./18]*18)

In [ ]:
class SeekingAgent:

  def __init__(self):
    '''
    Агент приближается к противнику
    '''
    pass

  def __call__(self, state):
    your_coords = get_boxer_coordinate(state, player = "you")
    their_coords = get_boxer_coordinate(state, player = "them")
    if (-10 <= your_coords[0] - their_coords[0] <= 10) and (abs(your_coords[1] - their_coords[1]) <= 10):
        return torch.tensor([0.] + [1./9] + [0.] * 8 + [1./9] * 8)
    else:
        if (your_coords[1] < their_coords[1]) and (your_coords[0] < their_coords[0]):
            return torch.tensor([0.] * 8 + [1.] + [0.] * 9)
        elif (your_coords[1] < their_coords[1]) and (your_coords[0] >= their_coords[0]):
            return torch.tensor([0.] * 6 + [1.] + [0.] * 11)
        elif (your_coords[1] >= their_coords[1]) and (your_coords[0] < their_coords[0]):
            return torch.tensor([0.] * 9 + [1.] + [0.] * 8)
        elif (your_coords[1] >= their_coords[1]) and (your_coords[0] >= their_coords[0]):
            return torch.tensor([0.] * 7 + [1.] + [0.] * 10)

In [ ]:
class PacifistAgent:

  def __init__(self):
    '''
    Агент случайно перемещается, но не бьет
    '''
    pass

  def __call__(self, state):
    return torch.tensor([1./10] + [0.] + [1./10] * 8 + [0.] * 8)


In [ ]:
# Играем агентами
with torch.no_grad():
  trajectories, rewards, renders = play_with_agents(env, 
                  {'first_0': SeekingAgent(), 'second_0': PacifistAgent()}, 
                  max_steps = 1000, 
                  render = True)
rewards

{'first_0': 20.999999999999982, 'second_0': -6.999999999999984}

In [ ]:
show_video(renders['first_0'])

In [ ]:
show_video(renders['second_0'])

# Награда


In [ ]:
def compute_total_reward(trajectory, discount = 0.9):
  R = np.array(trajectory["rewards"])
  for i in reversed(range(R.shape[0]-1)):
    R[i] += R[i+1] * discount
  trajectory['R'] = R
  return R


In [ ]:
compute_total_reward(trajectories['second_0'], 0.9)

array([0, 0, 0, ..., 0, 0, 0])

# Функция потерь

Мы для каждого состояния (первая колонка в таблице) мы можем расчитать вероятности каждого действия $p_i$ с точки зрения нашей модели, где $i$ -- это номер действия.

Пусть кумулятивная награда была $R$ (последняя колнка в таблице), тогда, если вероятность фактически предпринятого действия была $p_a$, то в функцию потерь для данного наблюдения мы относим: 
$$-R \cdot \log p_a$$

In [ ]:
def compute_trajectory_loss(trajectory, agent):
  loss = 0
  for i in range(len(trajectory['observations'])):
    p = agent(torch.FloatTensor(trajectory['observations'][i]))
    p_a = p[trajectory['actions'][i]]
    loss += -trajectory['R'][i] * torch.log(p_a)
  return loss

In [ ]:
compute_trajectory_loss(trajectories['second_0'], UnpredictableAgent())

tensor(-49.1363)

# Основной цикл обучения
Определим параметры обучения

In [ ]:
# Основной цикл обучения
NUM_EPISODES = 100 # количество эпизодов, которые будут сыграны для обучения
DISCOUNT = 0.9 # фактор дисконтирования
TRAIN_EVERY = 1

In [ ]:
import torch
import torchvision

model = torchvision.models.shufflenet_v2_x0_5(weights='IMAGENET1K_V1')
model.fc = torch.nn.Linear(model.fc.in_features, 18)
  

class BullyAgent_v2(torch.nn.Module):
  def __init__(self):
        super(BullyAgent_v2, self).__init__()
        self.my_model = model
        self.softmax = torch.nn.Softmax()
  
  def __call__(self, state):
    image = state.permute(2,0,1)
    image = image / 255.
    image = torch.unsqueeze(image[:, 30:-30,30:-30], dim = 0)
    image = self.my_model(image)
    image = torch.flatten(image)
    return self.softmax(image) + 0.2/18
    
agent = BullyAgent_v2()

Downloading: "https://download.pytorch.org/models/shufflenetv2_x0.5-f707e7126e.pth" to /root/.cache/torch/hub/checkpoints/shufflenetv2_x0.5-f707e7126e.pth


  0%|          | 0.00/5.28M [00:00<?, ?B/s]

In [ ]:
agent_last_version = BullyAgent_v2()
agent_last_version.load_state_dict(torch.load(PATH + "BullyAgent2_dict.pth"))
agent.load_state_dict(torch.load(PATH + "BullyAgent_v2_g2_dict.pth"))

<All keys matched successfully>

In [ ]:
from torch.optim import AdamW

agent.train()

optim = AdamW(agent.parameters())

trajectories = []
cur_total_reward = 0

for game in range(NUM_EPISODES):
  # Собираем траекторию
  if game % 2 == 0:
    trajs, total_rewards, _ = play_with_agents(
        env, 
        {'first_0': agent, 'second_0': agent_last_version},
        max_steps = 2500, device = None, render = False)
    # Получаем интересующую нас траекторию
    traj = trajs['first_0']
    cur_total_reward += total_rewards['first_0']
    # Расчитываем накопительную награду для этой траектории (колонка R)
    compute_total_reward(trajs['first_0'], discount = DISCOUNT)
    # Сохраняем траекторию
    trajectories.append(traj)

  else:
    trajs, total_rewards, _ = play_with_agents(
        env, 
        {'first_0': agent_last_version, 'second_0': agent},
        max_steps = 2500, device = None, render = False)
    # Получаем интересующую нас траекторию
    traj = trajs['second_0']
    cur_total_reward += total_rewards['second_0']
    # Расчитываем накопительную награду для этой траектории (колонка R)
    compute_total_reward(trajs['second_0'], discount = DISCOUNT)
    # Сохраняем траекторию
    trajectories.append(traj)

  if (game % TRAIN_EVERY) == 0:
    # Теперь непосредственно обучение.
    loss = 0
    for traj in trajectories:
      # В traj уже есть колонка R, т.е. колонка с накопительной наградой!
      loss += compute_trajectory_loss(traj, agent)
    
    optim.zero_grad()
    loss.backward()
    optim.step()

    print(f'Trained GAME {game}. Avg. Total Reward is {cur_total_reward / TRAIN_EVERY}.')
    trajectories = []
    cur_total_reward = 0

torch.save(agent.state_dict(), PATH + "BullyAgent_v2_g3_fixed_dict.pth")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Trained GAME 0. Avg. Total Reward is 29.999999999999645.
Trained GAME 1. Avg. Total Reward is 20.2.
Trained GAME 2. Avg. Total Reward is 28.569999999999887.
Trained GAME 3. Avg. Total Reward is 29.539999999999537.
Trained GAME 4. Avg. Total Reward is 30.57999999999983.
Trained GAME 5. Avg. Total Reward is 25.119999999999692.
Trained GAME 6. Avg. Total Reward is 26.229999999999702.
Trained GAME 7. Avg. Total Reward is 36.96999999999992.
Trained GAME 8. Avg. Total Reward is 36.969999999999544.
Trained GAME 9. Avg. Total Reward is 17.930000000000078.
Trained GAME 10. Avg. Total Reward is 35.22999999999944.
Trained GAME 11. Avg. Total Reward is 30.739999999999732.
Trained GAME 12. Avg. Total Reward is 3.4100000000000916.
Trained GAME 13. Avg. Total Reward is 24.429999999999875.
Trained GAME 14. Avg. Total Reward is 31.499999999999787.
Trained GAME 15. Avg. Total Reward is 31.839999999999712.
Trained GAME 16. Avg. Total Reward is 11.370000000000195.
Trained GAME 17. Avg. Total Reward is 31.

In [ ]:
# Играем агентами
with torch.no_grad():
  agent.eval()
  _, rewards, renders = play_with_agents( env, 
                  {'first_0': agent, 'second_0': agent_last_version}, 
                  max_steps = 7200, 
                  render = True)
rewards

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


{'first_0': 63.82999999999931, 'second_0': 59.82999999999972}

In [ ]:
show_video(renders['first_0'])

In [ ]:
with torch.no_grad():
  agent.eval()
  _, rewards, renders = play_with_agents( env, 
                  {'first_0': agent_last_version, 'second_0': agent}, 
                  max_steps = 7200, 
                  render = True)
rewards

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


{'first_0': 74.30999999999648, 'second_0': 76.30999999999648}

In [ ]:
show_video(renders['second_0'])